In [ ]:
import requests
import bs4 as bs
import pandas as pd
from tqdm import tqdm

In [ ]:
# api params and url - key needed from azure
# https://www.microsoft.com/en-us/bing/apis/bing-web-search-api

subscription_key = "<key goes here>"

search_url = "https://api.bing.microsoft.com/v7.0/news/search"
headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
params  = {"q": search_term, "textDecorations": True, "textFormat": "HTML", 'count':100, 'mkt': 'fr-FR'}

In [ ]:
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = json.dumps(response.json())

In [ ]:
def search(search_term):
    params["q"] = search_term
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    df = pd.json_normalize(response.json())
    df = pd.DataFrame(df['value'][0])
    return df

In [ ]:
companies = pd.read_csv('data/external/top_companies.txt', sep='	')

In [ ]:
all_data = []
for company in tqdm(companies['Entreprise']):
    df = search(company)
    df['company'] = company
    all_data.append(df)
    df.to_csv('dump/%s.csv' % company)

In [ ]:
all_articles = pd.concat(all_data)

In [ ]:
all_articles = pd.concat(all_data)
all_articles = all_articles.merge(
    companies[['Entreprise','Activité']],
    left_on='company', right_on='Entreprise'
    ).drop('Entreprise', axis=1)

In [ ]:
all_articles.to_csv('data/external/labelled articles.csv')

In [ ]:
import bs4 as bs

In [ ]:
tqdm.pandas()

# remove html characters
def clean_html(x):
    soup = bs.BeautifulSoup(x, 'html.parser')
    return soup.get_text()

In [ ]:
all_articles['name_clean'] = all_articles['name'].progress_apply(clean_html)
all_articles['clean_description'] = all_articles['description'].progress_apply(clean_html)
all_articles['naf_clean'] =  all_articles['Activité'].apply(lambda x: x.split('(')[-1].split(')')[0])

In [ ]:
mapping = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/7bb2184b-88cb-4c6c-a408-5a0081816dcd', sep=',')[['id_2', 'id_5']].set_index('id_5')

all_articles = all_articles.merge(mapping, left_on='naf_clean', right_index=True)

In [ ]:
clean_articles = all_articles[['company', 'name_clean', 'clean_description','id_2']]

In [ ]:
clean_articles.columns = ['company','title','text','naf']

In [ ]:
clean_articles.to_csv('data/external/labelled articles cleaned.csv')